# Setting up local AI code assist

I have been using a local llama3 LLM with ollama and the continue vs code extension with great results so far.

- install ollama
- install vs code extension
- configure continue extension to use ollama